In [1]:
import torch
import torch.nn as nn   
import torch.optim as optim
import numpy as np

In [2]:
setence = "Repeat is the best medicine for memory".split()
print(setence)

['Repeat', 'is', 'the', 'best', 'medicine', 'for', 'memory']


In [3]:
vocab = sorted(list(set(setence))) # sorted()를 사용하여 중복을 제거하고 정렬
print(vocab) # 단어의 순서로 정렬됨

['Repeat', 'best', 'for', 'is', 'medicine', 'memory', 'the']


In [4]:
vocab_size = len(vocab)
vocab_size 

7

In [7]:
word_to_index = {tkn:i for i, tkn in enumerate(vocab, 1)} # word를 리스트로 만들어 인덱스를 1부터 부여
print(word_to_index) # 각 단어에 대한 정수 인코딩 결과 출력
word_to_index['<unk>'] = 0 # <unk>는 OOV(Out-Of-Vocabulary) 단어를 의미. 단어 집합에 없는 단어들은 <unk> 토큰으로 인코딩
print(word_to_index) # <unk> 토큰을 고려한 단어 집합 출력

{'Repeat': 1, 'best': 2, 'for': 3, 'is': 4, 'medicine': 5, 'memory': 6, 'the': 7}
{'Repeat': 1, 'best': 2, 'for': 3, 'is': 4, 'medicine': 5, 'memory': 6, 'the': 7, '<unk>': 0}


In [10]:
print(word_to_index['for']) # 단어를 key로 하고 인덱스를 value로 하는 딕셔너리이므로 단어 'for'의 인덱스 출력
encode=[word_to_index[token] for token in setence]
print(encode) # 문장을 정수 인코딩한 결과 출력 

3
[1, 4, 7, 2, 5, 3, 6]


In [13]:
def build_data(sentence, word_to_index):
    encode=[word_to_index[t] for t in sentence]
    input_sequnce = encode[:-1] # 마지막 단어 제외하고 저장
    label_sequnce = encode[1:] # 첫 단어 제외하고 저장
    input_sequnce = torch.LongTensor(input_sequnce).unsqueeze(0) # 배치 차원 추가
    label_sequnce = torch.LongTensor([label_sequnce]) # label은 one-hot encoding 안 함 -> CrossEntropyLoss 사용
    print(input_sequnce)
    print(label_sequnce)
    return input_sequnce, label_sequnce


In [14]:
X,Y = build_data(setence, word_to_index) # 입력 시퀀스와 레이블 시퀀스 출력
print(X) # 입력 시퀀스 출력
print(Y) # 레이블 시퀀스 출력

tensor([[1, 4, 7, 2, 5, 3]])
tensor([[4, 7, 2, 5, 3, 6]])
tensor([[1, 4, 7, 2, 5, 3]])
tensor([[4, 7, 2, 5, 3, 6]])


In [19]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True): # def__init__ : 클래스의 초기화 함수 
        super(Net, self).__init__() # nn.Module 클래스의 속성들을 가지고 초기화
        
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=input_size) # 임베딩 층 생성. 단어 집합의 크기는 vocab_size이고 임베딩 벡터의 차원은 input_size
        self.rnn=nn.RNN(input_size, hidden_size, batch_first=batch_first) # RNN 층 생성. input_size는 임베딩 벡터의 크기이며, hidden_size는 은닉 상태의 크기
        self.fc=nn.Linear(hidden_size, vocab_size) # 출력층 생성. 은닉 상태의 크기는 hidden_size이며, 출력층의 뉴런은 단어 집합의 크기인 vocab_size

    def forward(self, x):
        output=self.embedding_layer(x) # 입력된 정수 인코딩을 임베딩 벡터로 변환
        #embedding layer: 단어를 임베딩 벡터로 만드는 역할. 크기변화(batch_size, seq_len, embedding_size)
        output, hidden = self.rnn(output) # RNN 층의 입력이자 임베딩 벡터를 RNN 층에 입력
        # RNN 층 : 은닉 상태의 크기를 hidden_size로 하여 RNN을 구현. 크기변화(batch_size, seq_len, hidden_size)
        # output: (batch_size, seq_len, hidden_size)
        # hidden: (1, batch_size, hidden_size) -> 마지막 시점의 은닉 상태 크기 

        output = self.fc(output)
        # 최종 출력층에서는 RNN 층의 출력을 받아서 단어 예측 -> 크기변화(batch_size, seq_len, vocab_size) => (batch_size * seq_len, vocab_size)

        return output.view(-1, output.size(2)) # 3D 텐서를 2D 텐서로 변환하여 리턴

In [20]:
vocab_size = len(word_to_index) # 단어 집합의 크기는 임베딩 층의 입력 차원
input_size = vocab_size
hidden_size = 20 # 은닉 상태의 크기는 20 


In [21]:
model=Net(vocab_size, input_size, hidden_size, batch_first=True) # 모델 생성
loss_function=nn.CrossEntropyLoss() # 손실 함수 생성
optimizer = optim.Adam(params=model.parameters()) # 옵티마이저 생성 

In [22]:
index_to_word = {v:k for k,v in word_to_index.items()} # 정수 인코딩 된 결과를 단어로 변환하기 위해 사용 
print(index_to_word)

{1: 'Repeat', 2: 'best', 3: 'for', 4: 'is', 5: 'medicine', 6: 'memory', 7: 'the', 0: '<unk>'}


In [23]:
decode = lambda y: [index_to_word.get(x) for x in y] # y에 대해서 단어로 변환하는 함수 생성 
print(decode)  # 람다 표현식을 사용하여 정수 시퀀스를 단어 시퀀스로 변환하는 함수 생성

<function <lambda> at 0x000001A603EAE0C0>


In [24]:
for step in range(200):
    optimizer.zero_grad()
    output=model(X) # 입력한 문장으로부터 예측된 문장 출력
    loss=loss_function(output, Y.view(-1)) # 손실 계산
    loss.backward() # 기울기 계산 
    optimizer.step() # 학습 진행 


    pred = output.softmax(-1).argmax(-1).tolist() # 예측된 결과는 원-핫 인코딩 형식 -> softmax 함수를 통해 확률로 변환 후 가장 높은 확률의 인덱스를 예측 값으로 설정 

    if step%10 == 0:
        print(step, pred, 'str:', ' '.join(['Repeat']+decode(pred))) # 10회 반복마다 예측 결과 출력


0 [3, 4, 3, 3, 1, 4] str: Repeat for is for for Repeat is
10 [6, 5, 3, 3, 1, 3] str: Repeat memory medicine for for Repeat for
20 [6, 5, 3, 3, 3, 2] str: Repeat memory medicine for for for best
30 [2, 5, 2, 5, 3, 6] str: Repeat best medicine best medicine for memory
40 [6, 5, 2, 5, 3, 6] str: Repeat memory medicine best medicine for memory
50 [6, 5, 2, 5, 3, 6] str: Repeat memory medicine best medicine for memory
60 [4, 5, 2, 5, 3, 6] str: Repeat is medicine best medicine for memory
70 [4, 5, 2, 5, 3, 6] str: Repeat is medicine best medicine for memory
80 [4, 7, 2, 5, 3, 6] str: Repeat is the best medicine for memory
90 [4, 7, 2, 5, 3, 6] str: Repeat is the best medicine for memory
100 [4, 7, 2, 5, 3, 6] str: Repeat is the best medicine for memory
110 [4, 7, 2, 5, 3, 6] str: Repeat is the best medicine for memory
120 [4, 7, 2, 5, 3, 6] str: Repeat is the best medicine for memory
130 [4, 7, 2, 5, 3, 6] str: Repeat is the best medicine for memory
140 [4, 7, 2, 5, 3, 6] str: Repeat is the